<a href="https://colab.research.google.com/github/GitSantiagopf/GenreRecognition/blob/main/GenreRecognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files
uploaded = files.upload()

In [ ]:
!unrar x canciones.rar


UNRAR 5.61 beta 1 freeware      Copyright (c) 1993-2018 Alexander Roshal


Extracting from canciones.rar


Would you like to replace the existing file canciones/Clasica/André Rieu - Ballade pour Adeline.mp3
     0 bytes, modified on 2023-04-19 01:43
with a new one
4966989 bytes, modified on 2023-04-19 01:43

[Y]es, [N]o, [A]ll, n[E]ver, [R]ename, [Q]uit A

Extracting  canciones/Clasica/André Rieu - Ballade pour Adeline.mp3        0%  OK 
Extracting  canciones/Clasica/Antonio Vivaldi La Stravaganza.mp3           1%  2%  3%  OK 
Extracting  canciones/Clasica/Beethoven – Pathetique Sonata 2nd Movement.mp3       4%  OK 
Extracting  canciones/Clasica/Beethoven – Sonata No.17 Tempest 3rd Movement.mp3       5%  6%  OK 
Extracting  canciones/Clasica/Brahms - Hungarian Dances No. 5 _ 6 conducted by Maciej Tomasiewicz.mp3       7%  8%  OK 
Extracting  canciones/Clasica/Chopin  Nocturne op9 No2.mp3                 9%  OK 
Extr

In [ ]:
import os
import soundfile as sf
from scipy.fft import fft
import numpy as np
import random

segment_size = 5 * 48000  # 5 segundos de audio
segments_per_song = 4  # 4 segmentos por canción

genres = ['Clasica', 'Pop', 'Metal']

song_spectra = {}
for genre in genres:
    song_spectra[genre] = []
    genre_dir = os.path.join('canciones', genre)
    for song_path in os.listdir(genre_dir):
        song, sample_rate = sf.read(os.path.join(genre_dir, song_path))
        song_length = len(song)
        for i in range(segments_per_song):
            segment_start = random.randint(0, song_length - segment_size)
            segment_end = segment_start + segment_size
            if segment_end > song_length:
                segment = np.zeros(segment_size)
                segment[:song_length - segment_start] = song[segment_start:]
            else:
                segment = song[segment_start:segment_end]
            spectrum = np.abs(fft(segment))
            song_spectra[genre].append(spectrum)

# Construir una base de datos de espectros de Fourier
spectra_db = {}
for genre in genres:
    spectra_db[genre] = np.mean(song_spectra[genre], axis=0)

# Clasificar una nueva canción
def classify_song(song_path):
    song, sample_rate = sf.read(song_path)
    song_length = len(song)
    segment_start_indices = random.sample(range(0, song_length - segment_size + 1, segment_size), segments_per_song)
    song_segments = []
    for start_idx in segment_start_indices:
        segment_end = start_idx + segment_size
        if segment_end > song_length:
            segment = np.zeros(segment_size)
            segment[:song_length - start_idx] = song[start_idx:]
        else:
            segment = song[start_idx:segment_end]
        song_segments.append(segment)
    song_spectra = []
    for segment in song_segments:
        spectrum = np.abs(fft(segment))
        song_spectra.append(spectrum)
    distances = {}
    for genre in genres:
        distances[genre] = np.mean([np.linalg.norm(spectrum - spectra_db[genre]) for spectrum in song_spectra])
    return min(distances, key=distances.get)


In [ ]:
genre = classify_song('Rels.mp3')
print(genre)

Pop
